# Get GPS from GPX

In [ ]:
import os
gps_path = '../gpssamples/'
#gpx_file = 'adour.gpx' 
gpx_file = '3145554.gpx'
gpx_path = os.path.join(gps_path,gpx_file)

In [ ]:
import gpxpy
import gpxpy.gpx 
gpx_file = open(gpx_path,'r',encoding='utf-8')
gpx = gpxpy.parse(gpx_file) # data from parsed gpx file

In [ ]:
''' gpsPointList function extract gps points from gpx file '''
''' gpxdata is a gpxpy object that returns data from a parsed gpx file'''
''' gpsPointList return a list of dictionnary points with Time, Lat, Long, Elev'''
def gpsPointList(gpxdata):
    point_list = []
    for track in gpxdata.tracks:
        for segment in track.segments: 
            for point in segment.points:
                point_info = {'Time':point.time,'Latitude':point.latitude,'Longitude':point.longitude,'Elevation':point.elevation}
                point_list.append(point_info)
    return point_list

In [ ]:
mypoints = gpsPointList(gpx)
for i in range(0,10):
    print(mypoints[i])

# Fill missing GPS

In [ ]:
import datetime
from datetime import datetime
from datetime import timedelta

def createTime(time):
    new_time = time
    new_time = new_time + timedelta(seconds=1)
    return new_time

def createLatitude(lat1,lat2):
    new_latitude = (lat1+lat2)/2
    new_latitude = round(new_latitude,6)
    return new_latitude

def createLongitude(long1,long2):
    new_longitude = (long1+long2)/2
    new_latitude = round(new_longitude,6)
    return new_longitude

def createElevation(elev1,elev2):
    new_elevation = (elev1+elev2)/2
    new_elevation = round(new_elevation,6)
    return new_elevation

In [ ]:
def fillGPS(inputGPSList,videoLength):
    filledGps = inputGPSList.copy()
    gps_length = len(filledGps)
    iteration_length = int((filledGps[gps_length-1]['Time'] - filledGps[0]['Time']).total_seconds())
    print(iteration_length)
    ## this section output a filled gps list of length iteration_length+1 = Delta T between last gps timestamp and first one
    i = 0
    while i < (iteration_length):
#        print(i)
        delta = filledGps[i+1]['Time']-filledGps[i]['Time']
        delta = int(delta.total_seconds())
        if delta > 1: # adding a newly created element at index i+1
            print(i)
            missing_time = createTime(filledGps[i]['Time'])
            missing_latitude = createLatitude(filledGps[i]['Latitude'],filledGps[i+1]['Latitude'])
            missing_longitude = createLongitude(filledGps[i]['Longitude'],filledGps[i+1]['Longitude'])
            missing_elevation = createElevation(filledGps[i]['Elevation'],filledGps[i+1]['Elevation'])
            new_gps = {'Time':missing_time,'Latitude':missing_latitude,'Longitude':missing_longitude,'Elevation':missing_elevation}
            filledGps.insert(i+1,new_gps)
        i = i+1
    ## this section add missing point at the end of the list, in case filledGps initial Delta time length is less than actual video length
    if len(filledGps) < videoLength:
        j = 0
        while len(filledGps) < videoLength:
            filledGps.insert(len(filledGps),filledGps[len(filledGps)-1])
            j = j+1

    return filledGps

In [ ]:
test = fillGPS(mypoints,615)
len(test)

# LongLat to shapely Point

In [ ]:
point0 = mypoints[0]
point0

In [ ]:
from shapely.geometry import Point
point = Point(0.0,0.0) #Longitude, Latitude
type(point)

In [ ]:
test1 = {'Time':point0['Time'],'the_geom':point,'Elevation':point0['Elevation']}
test1

In [ ]:
point1 = Point(point0['Longitude'],point0['Latitude'])
point1.coords.xy

In [ ]:
# convert gps Point to gps Shape Point
def longLat2Shape(gpsPoint):
    gpsShapePoint = {'Time':gpsPoint['Time'],'the_geom':Point(gpsPoint['Longitude'],gpsPoint['Latitude']),'Elevation':gpsPoint['Elevation']}
    return gpsShapePoint

In [ ]:
test = longLat2Shape(point0)
test

In [ ]:
test['the_geom'].coords.xy

In [ ]:
def gpsPoints2gpsShapePoints(gpsList):
    gpsShapeList = []
    for gpsPoint in gpsList:
        gpsShapePoint = longLat2Shape(gpsPoint)
        gpsShapeList.append(gpsShapePoint)
    return gpsShapeList

In [27]:
mygpsShapePoints = gpsPoints2gpsShapePoints(mypoints)
type(mygpsShapePoints[0]['the_geom'])

shapely.geometry.point.Point